In [235]:
# get data from mongo
from pymongo import MongoClient

from settings import *

# get the QRFA sequences
# col_name = 'meena/meena' msdialog_intent
col_name = 'qulac'


mongo = MongoClient()

def annotate_qa_strings(col_name):
    # parse meena subsets
    model_name = None
    if '/' in col_name:
        col_name, model_name = col_name.split('/')
        
    encoded_dialogues = []
    cmap = get_cmap(col_name)
    
    collection = mongo[DB_NAME][col_name]
    cursor = collection.find()
    for doc in cursor:
        if model_name:
            if doc['model_name'] != model_name:
                continue
        if 'ded_qrfa' in doc:
            qsra = doc['qrfa']  # qrfa qrfa_a qsra qrfa_m ded_qrfa ded_qrfa_m
#             print(qsra)
            encoded_dialogues.append('<'+qsra+'>')
#         else:
#             print(doc)
    return encoded_dialogues
#         print(doc[cmap['utterances']][-1])
#         break

ds = annotate_qa_strings(col_name)
print(len(ds))

10277


In [236]:
# absolute counts for unigrams
from collections import Counter

SKIP = ['B', 'G']

def find_ngrams(dialogues, n=2):
    # mine ngrams with a sliding window n (linear), e.g. QR, RQ
    ngrams = Counter()
    s = ''
    for d in 'o'.join(dialogues):
        if d == 'o':
            # reset pattern
            s = ''
            continue
        # skip duplicate symbols (loops)
        if not s or s[-1] != d:
            if d not in SKIP:
                s += d
        if len(s) == n:
            ngrams[s] += 1
            # slide over
            s = s[1:]
    return ngrams

_ngrams = find_ngrams(ds, n=1)
# print(_ngrams)

total = sum(_ngrams.values(), 0.0) - _ngrams['<'] * 2
# print(total)
# normalise counts to be relative to the total number of conver
das = {k: round(v / total, 2) for k, v in _ngrams.items() if k in ['Q', 'R', 'F', 'A']}
print(das)

{'Q': 0.44, 'R': 0.33, 'F': 0.23, 'A': 0.0}


In [237]:
# 1) count all bigrams
c_ngrams = find_ngrams(ds, n=2)

# filter out adjacent ones
# c_ngrams = {k: v for k, v in c_ngrams.items() if k not in ['AR', 'RA', 'FQ', 'QF']}

# total = sum(c_ngrams.values(), 0.0) - _ngrams['<'] * 2
_max = max(list(c_ngrams.values()))
# _max = max([v for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k])
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

# show boundaries
print('\n')
for n, c in c_ngrams.most_common():
     if '<' in n or '>' in n:
#     print([n, "%.2f"%(c / total)])
        print([n, "%.2f"%(c / len(ds))])

{'QR': 0.99, 'RF': 0.69, 'RQ': 0.31, 'FR': 0.01, 'QF': 0.0, 'QA': 0.0, 'AQ': 0.0, 'AF': 0.0}


['<Q', '0.99']
['F>', '0.69']
['Q>', '0.31']
['<F', '0.01']
['R>', '0.00']


In [221]:
# 1) count all bigrams
c_ngrams = find_ngrams(ds, n=3)

_max = max(list(c_ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

# show boundaries
print('\n')
for n, c in c_ngrams.most_common():
     if '<' in n or '>' in n:
        print([n, "%.2f"%(c/len(ds))])

{'FRF': 1.0, 'AQA': 0.98, 'RFR': 0.89, 'QAQ': 0.83, 'RFA': 0.58, 'AFA': 0.57, 'FAF': 0.48, 'FRQ': 0.47, 'FAQ': 0.47, 'QRF': 0.43, 'RQR': 0.41, 'QAF': 0.41, 'RQA': 0.36, 'AQR': 0.32, 'AFR': 0.29, 'QRQ': 0.28}


['<FR', '0.80']
['FA>', '0.45']
['FR>', '0.23']
['QA>', '0.21']
['<RF', '0.09']
['QR>', '0.07']
['<RQ', '0.05']
['AF>', '0.04']
['<FA', '0.03']
['<AQ', '0.01']
['<QR', '0.01']


In [27]:
c_ngrams = find_ngrams(ds, n=4)

_max = max(list(c_ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

{'QAQA': 1.0, 'AQAQ': 0.53, 'FAFA': 0.52, 'QAFA': 0.51, 'AQAF': 0.45, 'AFAF': 0.44, 'AFQA': 0.43, 'RFRF': 0.4, 'FRFA': 0.37, 'FARF': 0.37, 'QARF': 0.37, 'QAFQ': 0.35, 'FQAF': 0.35, 'ARFA': 0.33, 'FAQA': 0.32, 'RFAF': 0.29, 'FRFR': 0.27, 'RFAR': 0.24, 'RFQA': 0.23, 'FQAQ': 0.22, 'AFRF': 0.21, 'AFAQ': 0.2, 'FAFQ': 0.2, 'ARFR': 0.19, 'FQAR': 0.18, 'AFAR': 0.18, 'ARFQ': 0.18, 'FAFR': 0.16, 'AQFA': 0.16, 'QAFR': 0.15, 'QFAF': 0.14, 'QAQF': 0.14, 'FRAF': 0.13, 'AQAR': 0.13, 'QRFA': 0.12, 'RFAQ': 0.12, 'RAFA': 0.12, 'ARQA': 0.1, 'FAQF': 0.1, 'QARQ': 0.1, 'FRFQ': 0.09, 'QRFR': 0.09, 'QAQR': 0.09, 'AFRA': 0.08, 'QFAR': 0.08, 'FQFA': 0.08, 'QRQA': 0.08, 'RFRA': 0.08, 'RQAF': 0.08, 'RQAQ': 0.07, 'QRAF': 0.07, 'QFAQ': 0.07, 'ARAF': 0.07, 'FQRF': 0.07, 'AFQF': 0.07, 'RAFQ': 0.07, 'AQRF': 0.07, 'FARQ': 0.07, 'FARA': 0.07, 'AFQR': 0.06, 'QFRF': 0.06, 'QFQA': 0.06, 'QRFQ': 0.06, 'RAQA': 0.06, 'FRQA': 0.06, 'QARA': 0.06, 'FQRA': 0.05, 'RAFR': 0.05, 'RQFA': 0.05, 'ARQF': 0.05, 'RFQR': 0.05, 'RFQF': 0.05

In [28]:
c_ngrams = find_ngrams(ds, n=5)

_max = max(list(c_ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

{'AQAQA': 1.0, 'QAQAQ': 0.96, 'QAQAF': 0.68, 'QAFQA': 0.6, 'AFAFA': 0.51, 'FAFAF': 0.49, 'AQAFA': 0.46, 'FRFRF': 0.45, 'AFQAF': 0.4, 'RFRFA': 0.39, 'QAFAF': 0.39, 'FARFA': 0.37, 'RFARF': 0.36, 'FQAQA': 0.35, 'QARFA': 0.33, 'FRFAF': 0.31, 'FQAFA': 0.31, 'RFRFR': 0.3, 'AFAQA': 0.3, 'ARFRF': 0.29, 'RFAFA': 0.28, 'FAQAF': 0.27, 'FAFQA': 0.27, 'ARFQA': 0.26, 'FQARF': 0.26, 'AQAFQ': 0.26, 'FQAFQ': 0.26, 'ARFAR': 0.25, 'ARFAF': 0.25, 'QAFAQ': 0.23, 'AFQAQ': 0.23, 'AFARF': 0.23, 'QARFQ': 0.22, 'AFRFA': 0.21, 'FRFAR': 0.21, 'QAFRF': 0.2, 'FAFRF': 0.2, 'FARFR': 0.19, 'QARFR': 0.19, 'FAQAQ': 0.19, 'RFQAF': 0.19, 'QAFAR': 0.18, 'QAQFA': 0.18, 'AFQAR': 0.18, 'QAQAR': 0.17, 'FAFAQ': 0.17, 'AQARF': 0.17, 'RFAQA': 0.17, 'AFAFQ': 0.16, 'FAFAR': 0.15, 'FARFQ': 0.15, 'AQFAF': 0.15, 'QRFRF': 0.14, 'FRFQA': 0.14, 'FRAFA': 0.14, 'QFAFA': 0.14, 'RFAFR': 0.13, 'QARQA': 0.13, 'FAQFA': 0.13, 'AQAFR': 0.13, 'AFRFR': 0.13, 'RQAQA': 0.12, 'AFAFR': 0.12, 'RFQAR': 0.12, 'QFARF': 0.12, 'RFAFQ': 0.12, 'FRFAQ': 0.11, '

In [29]:
c_ngrams = find_ngrams(ds, n=6)

_max = max(list(c_ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

{'QAQAQA': 1.0, 'AQAQAQ': 0.55, 'AQAQAF': 0.37, 'QAQAFA': 0.37, 'FAFAFA': 0.31, 'QAFQAF': 0.3, 'RFRFRF': 0.29, 'AFAFAF': 0.26, 'FRFRFA': 0.26, 'QAFAFA': 0.25, 'AQAFQA': 0.24, 'FQAFQA': 0.23, 'QAQAFQ': 0.23, 'QAFAQA': 0.21, 'AFQAQA': 0.21, 'ARFARF': 0.21, 'RFARFA': 0.21, 'FRFRFR': 0.19, 'QARFQA': 0.19, 'QAFQAQ': 0.18, 'AFQAFA': 0.18, 'FAQAQA': 0.18, 'AQAFAF': 0.18, 'FRFARF': 0.18, 'QARFRF': 0.17, 'RFRFAF': 0.17, 'AFQAFQ': 0.16, 'FAQAFA': 0.16, 'FARFRF': 0.16, 'FQAQAF': 0.16, 'FARFAR': 0.16, 'FRFAFA': 0.16, 'FQAQAQ': 0.15, 'RFAFAF': 0.15, 'FARFAF': 0.15, 'ARFRFA': 0.14, 'AFAQAF': 0.14, 'QARFAF': 0.13, 'QAQARF': 0.13, 'ARFAFA': 0.13, 'FQAFAF': 0.13, 'FAFQAF': 0.13, 'RFRFAR': 0.13, 'QAFARF': 0.13, 'QAFQAR': 0.13, 'AFQARF': 0.12, 'FAFAQA': 0.12, 'FQARFA': 0.12, 'AFARFA': 0.12, 'ARFQAF': 0.12, 'AFAFQA': 0.12, 'AQAFAQ': 0.12, 'QARFAR': 0.12, 'AFRFAF': 0.11, 'FAFRFA': 0.11, 'AFRFRF': 0.11, 'FARFQA': 0.11, 'ARFRFR': 0.11, 'QAFRFA': 0.11, 'RFQARF': 0.1, 'FAFARF': 0.1, 'RFAFRF': 0.1, 'QAQAFR': 0.

In [30]:
c_ngrams = find_ngrams(ds, n=7)

_max = max(list(c_ngrams.values()))
ngrams = {k: round(v / _max, 2) for k, v in c_ngrams.most_common() if '<' not in k and '>' not in k}
print(ngrams)

{'AQAQAQA': 1.0, 'QAQAQAQ': 0.97, 'QAQAQAF': 0.6, 'QAQAFQA': 0.37, 'AQAQAFA': 0.36, 'FRFRFRF': 0.31, 'AFAFAFA': 0.3, 'FAFAFAF': 0.29, 'QAFQAQA': 0.29, 'AFQAFQA': 0.27, 'FQAQAQA': 0.26, 'RFRFRFA': 0.26, 'QAQAFAF': 0.25, 'QAFQAFA': 0.24, 'QAFQAFQ': 0.23, 'FARFARF': 0.23, 'RFRFRFR': 0.22, 'ARFARFA': 0.22, 'AQAFAFA': 0.21, 'FQAFQAF': 0.21, 'QAFAFAF': 0.2, 'AQAFAQA': 0.19, 'AQAFQAF': 0.19, 'AQAQAFQ': 0.18, 'RFRFARF': 0.18, 'ARFRFRF': 0.18, 'FRFRFAF': 0.18, 'AFQAQAF': 0.17, 'AFAQAFA': 0.17, 'QARFARF': 0.17, 'FRFARFA': 0.17, 'RFARFAR': 0.17, 'QAFAQAF': 0.16, 'QAQAFAQ': 0.16, 'QAFQARF': 0.16, 'FARFRFA': 0.16, 'FRFAFAF': 0.15, 'RFARFRF': 0.15, 'FQAQAFA': 0.15, 'AFAQAQA': 0.15, 'FQARFQA': 0.14, 'RFRFAFA': 0.14, 'AQAFQAQ': 0.14, 'RFAFAFA': 0.14, 'FARFAFA': 0.14, 'FRFRFAR': 0.14, 'AFQAQAQ': 0.14, 'FAQAQAQ': 0.13, 'QARFQAF': 0.13, 'QARFRFA': 0.13, 'FQAFAFA': 0.13, 'FAQAQAF': 0.13, 'QAQAFRF': 0.12, 'AFRFRFA': 0.12, 'QAFAFQA': 0.12, 'ARFQARF': 0.12, 'AFQAFAF': 0.12, 'ARFAFAF': 0.12, 'QAFAQAQ': 0.12, 